# Preprocessing


## All preprocessing just needs to be done once. 
### If already done go direclty to DA patterns to load csv


In [1]:
import os
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import re
from nltk.tokenize import word_tokenize

#### Open all datasets from different tasks

In [ ]:
df_t1 = pd.read_json('./data_TM2/flights.json')
df_t2 = pd.read_json('./data_TM2/food-ordering.json')
df_t3 = pd.read_json('./data_TM2/hotels.json')
df_t4 = pd.read_json('./data_TM2/movies.json')
df_t5 = pd.read_json('./data_TM2/music.json')
df_t6 = pd.read_json('./data_TM2/restaurant-search.json')
df_t7 = pd.read_json('./data_TM2/sports.json')

# create extra column in the beginning for type of task before merging
df_t1.insert(0, 'task', 'flights')
df_t2.insert(0, 'task', 'food-ordering')
df_t3.insert(0, 'task', 'hotels')
df_t4.insert(0, 'task', 'movies')
df_t5.insert(0, 'task', 'music')
df_t6.insert(0, 'task', 'restaurant-search')
df_t7.insert(0, 'task', 'sports')


#### Normalize json and change to pandas 

In [ ]:
def process_raw(df): #, name):
    dialogs = []

    for e,i in enumerate(df['utterances']):
        for j in i:
            new_df=pd.json_normalize(j)
            new_df.insert(0, 'task', df['task'][e])
            new_df.insert(1, 'conversation_id', df['conversation_id'][e])
            new_df.insert(2, 'instruction_id', df['instruction_id'][e])
        
            dialogs.append(new_df)

    large_df = pd.concat(dialogs, ignore_index=True)
#   large_df.to_csv('./data_TM2/processed_utterances_'+name+'.csv')
    
    return large_df

#### Create unique dataset with normalized tasks

In [ ]:
def unique(df_list):
    large_frames = []

    for df_ in df_list:
        large_df = process_raw(df_)
        large_frames.append(large_df)

    all_tasks = pd.concat(large_frames, ignore_index=True)
    all_tasks.to_csv('./data_TM2/concatenated_tasks.csv')
    
    return all_tasks

In [ ]:
df_list = [df_t1, df_t2, df_t3, df_t4, df_t5, df_t6, df_t7]
all_tasks = unique(df_list)

In [ ]:
# # #essa parte é de antes porem com split incluindo simbolo do split
# new_text = []
# for ut in df['text']:
#     new_ut = re.split('(\.)', ut)
#     new_text.append(new_ut)
# df['new_text'] = new_text

# #to keep delimiter in sentence
# #not very efficient way of doing it, apparently a better way using regex symbols

# new_lista = []
# for row in df['new_text']:
# #     print(row)
#     for e, word in enumerate(row):
#         if word == '.':
#             new_lista.append(row[e-1]+row[e])
#             new_lista.remove(row[e-1])
#         else:
#             new_lista.append(word)

# df = df.explode('new_text')
# df = df.reset_index(drop=True)

In [ ]:
from nltk import sent_tokenize

df = pd.read_csv('./data_TM2/concatenated_tasks.csv', index_col=0)

tokenized = []
for ut in df['text']:
    tokenized.append(sent_tokenize(ut))
    
df['new_lista'] = tokenized
df = df.explode('new_lista')
df = df.reset_index(drop=True)
df.head(20)

## Load whole dataset to work with DA patterns

In [2]:
df = pd.read_csv('./data_TM2/processed_utterances_all_tasks.csv', index_col=0)

### Find substrings
make list of words and rules for labeling DA
https://towardsdatascience.com/check-for-a-substring-in-a-pandas-dataframe-column-4b949f64852#:~:text=Using%20%E2%80%9Ccontains%E2%80%9D%20to%20Find%20a,substring%20and%20False%20if%20not.

- add new column for subset. since each utterance can only be one thing one column for all DA should be enough. 
- Later I'll have to check the vocabulary used for generic rules (FS_base_greeting or whatever is)
- Also study in the UX book what is repair and other patterns:
    - page 243 to 246: NCF pattern language summary

Tips:
- #NOT_INCLUDE_WORD: 
    - not_fifa = football_soccer_games.loc[~football_soccer_games['Name'].str.contains('FIFA')]
- #to make a new dataframe with slicing subset:
    - new_df = df.loc[df['text'].str.contains('|'.join(end_of_request), case=False)]

In [ ]:
# DA indicators. Obs: the vars bellow are sensitive to spacing !!! 'hi' != ' hi'

greeting = [' hi','hi.','hello','bye'] 
repair_initiator = ['sorry','repeat', 'understand', 'mean'] 
request_summary = ['correct?','confirm'] #when you ask an information to be confirmed
confirmation = ['yes','correct.','correct ', 'that\'s it', 'indeed'] #confirmation é a resposta que confirma: ex yes | correct | that's it
sentence_closer = ['awesome','great','perfect','exactly','that\'s all','that is all', 'thanks', 'thank you', 'pleasure', 'excellent'] 
question = ['\?']
receipt = ['You are welcome', 'you\'re welcome']
grant = ['got it', 'sure']

## !!!!! order-sensitive > hierarchical!!!! this variable chooses only one label, rules goes on order. once it assigns one rule it won't check next

temp2=df.new_text.fillna("0")
df['DA_indicators_sent'] =  np.where(temp2.str.contains('|'.join(repair_initiator), case=False), "repair_initiator",
                            np.where(temp2.str.contains('|'.join(greeting), case=False), "greeting",
                            np.where(temp2.str.contains('|'.join(request_summary), case=False),  "request_summary",
                            np.where(temp2.str.contains('|'.join(confirmation), case=False),  "confirmation",
                            np.where(temp2.str.contains('|'.join(receipt), case=False),  "receipt",
                            np.where(temp2.str.contains('|'.join(grant), case=False),  "grant",
                            np.where(temp2.str.contains('|'.join(question), case=False),  "question",
                            np.where(temp2.str.contains('|'.join(sentence_closer), case=False), "sentence_closer", "x"))))))))

df['DA_indicators_sent'].value_counts()

#it works! #binary variables and in the end gather in one new column

df['DA_rep'] =  np.where(temp2.str.contains('|'.join(repair_initiator), case=False), "repair_initiator", '')
df['DA_greet'] =  np.where(temp2.str.contains('|'.join(greeting), case=False), "greeting", '')
df['DA_req_sum'] =  np.where(temp2.str.contains('|'.join(request_summary), case=False), "request_summary", '')
df['DA_conf'] =  np.where(temp2.str.contains('|'.join(confirmation), case=False), "confirmation", '')
df['DA_receipt'] =  np.where(temp2.str.contains('|'.join(receipt), case=False), "receipt", '')
df['DA_grant'] =  np.where(temp2.str.contains('|'.join(grant), case=False), "grant", '')
df['DA_quest'] =  np.where(temp2.str.contains('|'.join(question), case=False), "question", '')
df['DA_closer'] =  np.where(temp2.str.contains('|'.join(sentence_closer), case=False), "sentence_closer", '')
df['all_DA'] = df[['DA_rep', 'DA_greet','DA_req_sum', 'DA_conf', 'DA_receipt', 'DA_grant', 'DA_quest', 'DA_closer']].agg(' '.join, axis=1)

df['all_DA'] = [word_tokenize(da) for da in df['all_DA']]

# df.loc[:, ['speaker', 'new_text', 'all_DA']].to_csv('analysis_multiple_DA_cases.csv')
df.loc[:, ['speaker', 'new_text', 'all_DA']]
counts = df['all_DA'].value_counts()

# counts.to_csv('count_overlaping_DAs.csv')
data = {'counts':counts}
df_counts = pd.DataFrame(data)
df_counts.head(45)

## Second iteration

In [57]:
# DA indicators. Obs: the vars bellow are sensitive to spacing !!! 'hi' != ' hi'

greeting = [' hi','hi.','hello','yo ', 'hey'] 
repair_initiator = ['sorry','repeat', 'understand', 'mean ', 'what?', 'example?']
repair = ['I meant']
request_summary = [' correct?','confirm']  
confirmation = ['yes',' correct.',' correct ', 'that\'s it', 'indeed', 'yeah', 'that\'s right', 'you got it', 'yep', 'sure', 'okay', 'all right'] 
sequence_closer = ['awesome','great','perfect','exactly','that\'s all','that is all', 'thanks', 'thank you', 'pleasure', 'excellent', 'okay', 'excellent', 'too bad', 'oh well']
inquiry = ['\?']
receipt = ['You are welcome', 'you\'re welcome']
disconfirmation = [' no ', 'wrong', 'incorrect', 'not really']

## !!!!! order-sensitive !!!!

#this variable chooses only one label, rules goes on order. once it assigns one rule it won't check next
temp2=df.new_text.fillna("0")
df['DA_indicators_sent'] =  np.where(temp2.str.contains('|'.join(repair_initiator), case=False), "repair_initiator",
                            np.where(temp2.str.contains('|'.join(greeting), case=False), "greeting",
                            np.where(temp2.str.contains('|'.join(request_summary), case=False),  "request_summary",
                            np.where(temp2.str.contains('|'.join(confirmation), case=False),  "confirmation",
                            np.where(temp2.str.contains('|'.join(receipt), case=False),  "receipt",
                            np.where(temp2.str.contains('|'.join(disconfirmation), case=False),  "disconfirmation",
                            np.where(temp2.str.contains('|'.join(disconfirmation), case=False),  "disconfirmation",
                            np.where(temp2.str.contains('|'.join(repair), case=False),  "repair",
                            np.where(temp2.str.contains('|'.join(sequence_closer), case=False), "sequence_closer", "x")))))))))

print(df['DA_indicators_sent'].value_counts())


#it works! #binary variables and in the end gather in one new column
##### ainda n sei se repair solo fica ######                                     
                                     
df['DA_rep_init'] =  np.where(temp2.str.contains('|'.join(repair_initiator), case=False), "repair_initiator", '')
df['DA_rep'] =  np.where(temp2.str.contains('|'.join(repair), case=False), "repair", '')
df['DA_greet'] =  np.where(temp2.str.contains('|'.join(greeting), case=False), "greeting", '')
df['DA_req_sum'] =  np.where(temp2.str.contains('|'.join(request_summary), case=False), "request_summary", '')
df['DA_conf'] =  np.where(temp2.str.contains('|'.join(confirmation), case=False), "confirmation", '')
df['DA_receipt'] =  np.where(temp2.str.contains('|'.join(receipt), case=False), "receipt", '')
df['DA_disconf'] =  np.where(temp2.str.contains('|'.join(disconfirmation), case=False), "disconfirmation", '')
df['DA_inquiry'] =  np.where(temp2.str.contains('|'.join(inquiry), case=False), "inquiry", '')
df['DA_closer'] =  np.where(temp2.str.contains('|'.join(sequence_closer), case=False), "sequence_closer", '')
df['all_DA'] = df[['DA_rep_init', 'DA_rep', 'DA_greet','DA_req_sum', 'DA_conf', 'DA_receipt', 'DA_disconf', 'DA_inquiry', 'DA_closer']].agg(' '.join, axis=1)

df['all_DA'] = [word_tokenize(da) for da in df['all_DA']]

df.loc[:, ['speaker', 'new_text', 'all_DA']].to_csv('analysis_multiple_DA_cases_2nditeration.csv')
df.loc[:, ['speaker', 'new_text', 'all_DA']]
counts = df['all_DA'].value_counts()

counts.to_csv('count_overlaping_DAs_2nditeration.csv')
data = {'counts':counts}
df_counts = pd.DataFrame(data)
df_counts.head()

x                   215931
greeting             80406
confirmation         68747
sequence_closer      33260
repair_initiator     30590
receipt               4380
request_summary       2233
disconfirmation       1324
repair                  12
Name: DA_indicators_sent, dtype: int64


,counts
[],166900
[inquiry],49031
[greeting],46156
"[confirmation, sequence_closer]",39767
[sequence_closer],32785


## Now sample a part to annotate
### criteria:

- take from all taks
- annotate full dialogues
- how many dialogues are there? what's the percentage of full dialogues i want to annotate?
17304 dialogues (index 0 demarks the begining of a dialogue)

In [64]:
#number of dialogues:
df_count = pd.read_csv('./data_TM2/concatenated_tasks.csv', index_col=0)
print(df_count['conversation_id'].nunique())

#average size of dialog
print(df_count['index'].mean())

#dialogues per task (i.e True column)
pd.crosstab(df_count['task'], df_count['index'] ==0).sort_values(by='task', ascending=False)

17289
11.578471098680227


index,False,True
task,,
sports,45099,3481
restaurant-search,62929,3276
music,25309,1603
movies,56701,3056
hotels,60691,2357
food-ordering,12903,1050
flights,60865,2481


In [136]:
#sample randomly 167 dialogues, that will be split into 2 batches:
#selection made using the unique conversation_ids

unique_conv_id = df['conversation_id'].unique()
sample_ids = np.random.choice(unique_conv_id, 172)

sample_b1  = []
sample_b2  = []
for e, row in enumerate(df['conversation_id']):
    for b1 in sample_ids[:int(len(sample_ids)/2)]:
        if row == b1:
            sample_b1.append(df.loc[e,:])
    for b2 in sample_ids[int(len(sample_ids)/2):]:
        if row == b2:
            sample_b2.append(df.loc[e,:])

df_sample_b1 = pd.DataFrame(sample_b1)
df_sample_b2 = pd.DataFrame(sample_b2)

df_sample_b1.to_csv('./data_TM2/sample_b1.csv')
df_sample_b2.to_csv('./data_TM2/sample_b2.csv')

## Getting insights from k-means

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['new_text'])

true_k = 5
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

for i in range(true_k):
    print('Cluster %d:' % i),
    for ind in order_centroids[i, :5]:
        print(' %s' % terms[ind])

In [ ]:
print('Prediction')
X = vectorizer.transform(['hi, can you help me?'])
predicted = model.predict(X)
print(predicted)

## Repair cases

Next round to detecting misunderstanding repair happens


In [ ]:
#THIS WORKS! need to apply all filters
# df.loc[df['column name'] condition, 'new column name'] = 'value if condition is met'

df.loc[(df['speaker'] != df['speaker'].shift(1)) & (df['DA_indicators_sent'].shift(1) == 'repair_initiator'), 'teste_col2'] = 'SECOND'

df['teste_col2'].value_counts()

In [ ]:
df.loc[df['teste_col2'] == 'SECOND']
#second is more an ANSWER (i.e. next round) to: sorry, what did you say? OR what did you mean?

In [ ]:
print('a case of self repair: ')
print(df.iloc[1708,4:-1])
print('\n\n')
print(df.iloc[1709,4:-1])

## Cosine similarity for paraphrasing

In [ ]:
a= 'he is the king of the world'
b= 'His majesty rules the planet'
c= 'I love my dog'


